# LLaMA Factory Colab Tutorial

Please use a **free** Tesla T4 Colab GPU to run this!

Project homepage: https://github.com/hiyouga/LLaMA-Factory

In [1]:
%%capture
!git clone https://github.com/hiyouga/LLaMA-Factory.git
!pip install /kaggle/working/LLaMA-Factory
!pip install /kaggle/working/LLaMA-Factory[bitsandbytes]
!pip install "unsloth[cu121-torch211] @ git+https://github.com/unslothai/unsloth.git"

# Temporary fix for https://github.com/huggingface/datasets/issues/6753
# !pip install datasets==2.16.0 fsspec==2023.10.0 gcsfs==2023.10.0

In [2]:
import torch
try:
  assert torch.cuda.is_available() is True
except AssertionError:
  print("Please set up a GPU before using LLaMA Factory: https://medium.com/mlearning-ai/training-yolov4-on-google-colab-316f8fff99c6")

In [3]:
# Authorize huggingface
!python -c "from kaggle_secrets import UserSecretsClient; from huggingface_hub import HfFolder; hftoken = UserSecretsClient().get_secret('hf-token'); HfFolder().save_token(hftoken)"

## Fine-tune model via Command Line

In [4]:
# gc.collect()
torch.cuda.empty_cache()

In [5]:
# !rm -r /kaggle/working/llama2
import torch
print("PyTorch Version:", torch.__version__)
print("CUDA Version:", torch.version.cuda)

PyTorch Version: 2.1.1+cu121
CUDA Version: 12.1


In [6]:
def get_latest_checkpoint(output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        return None
    dirs = os.listdir(output_dir)
    dirs = [d for d in dirs if d.startswith("checkpoint")]
    dirs = sorted(dirs, key=lambda x: int(x.split("-")[1]))
    path = dirs[-1] if len(dirs) > 0 else None
    return os.path.join(output_dir, path) if path is not None else None

In [7]:
import os
from huggingface_hub import snapshot_download
# Set environment variables
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'
os.environ["WANDB_DISABLED"] = "true"
os.environ['TOKENIZERS_PARALLELISM'] = "false"
# model and data
# model_name = "unsloth/mistral-7b-instruct-v0.2-bnb-4bit"
model_name = "unsloth/llama-2-7b-bnb-4bit"
# model_name = "unsloth/gemma-7b-bnb-4bit"
dataset_dir = "/kaggle/input/json-arxiv"
# Train set
dataset_name = "train_data_25K_json"
# Test set
# dataset_name = "test_data_1K_json"
template = "alpaca"

# File
model = "llama2"
output_dir = f"/kaggle/working/{model}"
logging_dir = f"/kaggle/working/{model}/logs"
tokenized_path = f"/kaggle/working/{model}/tokenized_dataset"
hub_model_id = f"BrijeshGiri/llm-ds-{model}"
# Download HF repo
try:
    snapshot_download(repo_id=hub_model_id, local_dir=output_dir, local_dir_use_symlinks=False)
except:
    print("HF repo does not exist. Starting Fresh training.")
    
resume_from_checkpoint = get_latest_checkpoint(output_dir)

# hyperparams
learning_rate = 1e-5
num_epochs = 5
warmup_ratio = 1/num_epochs
lr_sched = "reduce_lr_on_plateau"
optim = "adamw_bnb_8bit"

# bs
per_device_train_batch_size = 2 # 1 for gemma
per_device_eval_batch_size = 2 # 1 for gemma
gradient_accumulation_steps = 2
cutoff = 2048 # input token cutoff length
dataloader_num_workers = 4
preprocessing_num_workers = 4
# rope_scaling = "dynamic" # {linear,dynamic,None} - for making normal model work with longer context in a simpler way

# steps
eval_steps = 100
save_steps = 100
logging_steps = 50
save_total_limit = 20

#lora
lora_target = "q_proj,k_proj,v_proj,o_proj,gate_proj,up_proj,down_proj"
lora_rank = 8
lora_alpha = 8

#removed         
# Construct the command using f-string
command = f"""
accelerate launch \
    --num_processes=2 \
    /kaggle/working/LLaMA-Factory/src/train_bash.py \
    --stage sft \
    --do_train \
    --model_name_or_path {model_name} \
    --dataset_dir {dataset_dir} \
    --dataset {dataset_name} \
    --template {template} \
    --finetuning_type lora \
    --lora_target {lora_target} \
    --lora_rank {lora_rank} \
    --lora_alpha {lora_alpha} \
    --output_dir {output_dir} \
    --overwrite_cache \
    --overwrite_output_dir \
    --cutoff_len {cutoff} \
    --preprocessing_num_workers {preprocessing_num_workers} \
    --dataloader_num_workers {dataloader_num_workers} \
    --per_device_train_batch_size {per_device_train_batch_size} \
    --per_device_eval_batch_size {per_device_eval_batch_size} \
    --gradient_accumulation_steps {gradient_accumulation_steps} \
    --lr_scheduler_type {lr_sched} \
    --warmup_ratio {warmup_ratio} \
    --logging_steps {logging_steps} \
    --logging_dir {logging_dir} \
    --save_steps {save_steps} \
    --eval_steps {eval_steps} \
    --save_total_limit {save_total_limit} \
    --evaluation_strategy steps \
    --load_best_model_at_end \
    --learning_rate {learning_rate} \
    --optim {optim} \
    --num_train_epochs {num_epochs} \
    --val_size 0.005 \
    --ddp_timeout 180000000 \
    --plot_loss \
    --fp16 \
    --use_unsloth \
    --quantization_bit 4 \
    --packing True \
    --report_to tensorboard \
    --tokenized_path {tokenized_path} \
    --push_to_hub \
    --hub_model_id {hub_model_id} \
    --hub_strategy all_checkpoints \
"""
# Debugging --max_samples 32
if resume_from_checkpoint is not None:
    command += f"""--resume_from_checkpoint {resume_from_checkpoint} \
    """
# Execute tokenization command
!{command} 
!{command} # Backup incase of tokenization

Fetching 340 files:   0%|          | 0/340 [00:00<?, ?it/s]

checkpoint-100/adapter_config.json:   0%|          | 0.00/732 [00:00<?, ?B/s]

checkpoint-100/README.md:   0%|          | 0.00/5.10k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/732 [00:00<?, ?B/s]

...

 35%|███████████▏                    | 4763/13570 [11:49:33<48:09:58, 19.69s/it]

 35%|███████████▏                    | 4764/13570 [11:49:53<48:19:22, 19.76s/it]

 35%|███████████▏                    | 4765/13570 [11:50:13<48:21:47, 19.77s/it]

 35%|███████████▏                    | 4766/13570 [11:50:33<48:31:23, 19.84s/it]

 35%|███████████▏                    | 4767/13570 [11:50:52<48:20:28, 19.77s/it]

### Infer the fine-tuned model

In [ ]:
# from llmtuner import ChatModel
# chat_model = ChatModel(dict(
#   model_name_or_path="Qwen/Qwen1.5-0.5B-Chat",
#   adapter_name_or_path="test_identity", # output dir of our training
#   finetuning_type="lora",
#   template="qwen", # change to alpaca
# ))
# messages = []
# while True:
#   query = input("\nUser: ")
#   if query.strip() == "exit":
#     break
#   if query.strip() == "clear":
#     messages = []
#     continue

#   messages.append({"role": "user", "content": query})
#   print("Assistant: ", end="", flush=True)
#   response = ""
#   for new_text in chat_model.stream_chat(messages):
#     print(new_text, end="", flush=True)
#     response += new_text
#   print()
#   messages.append({"role": "assistant", "content": response})

### Merge LoRA weights

In [ ]:
# from llmtuner import export_model
# export_model(dict(
#   model_name_or_path="Qwen/Qwen1.5-0.5B-Chat",
#   adapter_name_or_path="test_identity",
#   finetuning_type="lora",
#   template="qwen",
#   export_dir="test_exported",
#   # export_hub_model_id="your_hf_id/test_identity",
# ))